# Trabalho 2: Redes Convolucionais

### Alunos
- **Arthur de Sá Antero - 212006577**
- **Arthur Mota Furtado - 200014935**

Neste trabalho, iremos aplicar algumas redes convolucionais para a tarefa de classificação de imagens. Como base de treinamento, usaremos um conjunto de imagens de flores, que serão classificadas em 5 classes: tulipa, girassol, rosa, dente-de-leão e margarida.

Inicialmente treinaremos um simples rede MLP com uma camada oculta, e depois iremos testar diferentes tipos de redes convolucionais pré-treinadas, que colocaremos antes das camadas densas da rede MLP.


In [16]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Carregamento dos dados

Aqui carregamos os dados diretamente da biblioteca tensorflow_datasets, e normalizamos os dados para o formato que o modelo espera. (imagens de 150x150, com 3 canais, float32)

In [3]:
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[70%:]"], ## Train test split
    batch_size=-1,
    as_supervised=True, # Include labels
)
## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))
## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)
print (train_ds.shape)
print (test_ds.shape)

2024-12-22 10:49:31.667909: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-22 10:49:31.782603: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-12-22 10:49:31.782884: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2611200000 Hz
2024-12-22 10:49:32.729927: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3488249856 exceeds 10% of free system memory.
2024-12-22 10:49:34.207193: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 932238720 exceeds 10% of free system memory.


(2569, 150, 150, 3)
(1101, 150, 150, 3)


### Declaração do modelo MLP

Aqui temos os a declaração do modelo MLP, que é um modelo de rede neural multi-camada, com uma função de ativação sigmoide e uma função de regressão linear. Ele utilizará duas camadas de 128 neurônios, sendo ambas ativadas com a função relu.

In [4]:
model = models.Sequential([
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(5, activation='softmax')
])

In [5]:

from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

Epoch 1/50
65/65 [==============================] - 3s 47ms/step - loss: 509.4144 - accuracy: 0.2533 - val_loss: 158.3735 - val_accuracy: 0.3521
Epoch 2/50
65/65 [==============================] - 3s 44ms/step - loss: 127.3345 - accuracy: 0.3652 - val_loss: 64.5273 - val_accuracy: 0.3813
Epoch 3/50
65/65 [==============================] - 3s 47ms/step - loss: 49.8208 - accuracy: 0.3930 - val_loss: 34.9322 - val_accuracy: 0.3346
Epoch 4/50
65/65 [==============================] - 3s 49ms/step - loss: 27.6560 - accuracy: 0.4279 - val_loss: 27.3815 - val_accuracy: 0.3268
Epoch 5/50
65/65 [==============================] - 3s 48ms/step - loss: 18.5574 - accuracy: 0.4149 - val_loss: 23.0015 - val_accuracy: 0.2763
Epoch 6/50
65/65 [==============================] - 3s 48ms/step - loss: 13.0571 - accuracy: 0.4567 - val_loss: 12.3043 - val_accuracy: 0.3735
Epoch 7/50
35/35 [==============================] - 0s 11ms/step - loss: 453.4857 - accuracy: 0.3488
Accuracy: 0.3487738370895386


Ao treinarmos o modelo, percebemos que a acurácia do modelo é muito abaixo de um valor aceitável, com o modelo errando na maioria dos casos. Isso dá pela natureza de lidar com imagens mais complexas, que modelos não convolucionais tem uma maior dificuldade em generalizar.

Para resolver esse problema, utilizaremos uma outra rede convolucional já pré-treinada, que é o VGG16, como base para o modelo. Essa rede é treinada para classificar imagens de alta resolução, e é capaz de generalizar bem para imagens de baixa resolução.

In [6]:
## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights
## Preprocessing input
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

Depois da saída da rede VGG16, treinaremos um topo uma nova camada MLP bem simples, que possa classificar imagens de maneira mais precisa. Como já temos muito processamento dentro da rede VGG16, podemos usar camadas com menos neurônios e com menos parâmetros.

In [7]:
convolutional_model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(5, activation="softmax")
])

In [8]:
convolutional_model.compile(
optimizer='adam',
loss='categorical_crossentropy',
metrics=['accuracy'],
)
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)
convolutional_model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

# Evaluate the model on the test dataset
loss, accuracy = convolutional_model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

Epoch 1/50
65/65 [==============================] - 126s 2s/step - loss: 2.9836 - accuracy: 0.4560 - val_loss: 1.2050 - val_accuracy: 0.5817
Epoch 2/50
65/65 [==============================] - 131s 2s/step - loss: 0.6830 - accuracy: 0.7455 - val_loss: 1.1780 - val_accuracy: 0.6206
Epoch 3/50
65/65 [==============================] - 131s 2s/step - loss: 0.4452 - accuracy: 0.8288 - val_loss: 1.1863 - val_accuracy: 0.6420
Epoch 4/50
65/65 [==============================] - 132s 2s/step - loss: 0.2822 - accuracy: 0.9120 - val_loss: 1.1315 - val_accuracy: 0.6887
Epoch 5/50
65/65 [==============================] - 132s 2s/step - loss: 0.1585 - accuracy: 0.9485 - val_loss: 1.2175 - val_accuracy: 0.6868
Epoch 6/50
65/65 [==============================] - 133s 2s/step - loss: 0.1115 - accuracy: 0.9609 - val_loss: 1.2734 - val_accuracy: 0.6887
Epoch 7/50
65/65 [==============================] - 134s 2s/step - loss: 0.0809 - accuracy: 0.9724 - val_loss: 1.3385 - val_accuracy: 0.6965
Epoch 8/50
65

Como podemos ver, mesmo com a rede VGG16, o resultado não é muito bom. O que ressalta o quão difícil é o dataset de treinamento, pois o dataset tem muito dados de treinamento, e as imagens em si são muito diferentes. Para melhorarmos o resultados, tentaremos usar uma outra rede neural, e outras técnicas de melhoria do aprendizado de máquina, para evitar pontos ótimos locais.

In [9]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input


## Loading ResNet50 model
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights
## Preprocessing input
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

print(base_model.output_shape)


87916544/87910968 [==============================] - 4s 0us/step
(None, 3, 3, 2048)


In [10]:
## Training
convolutional_model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(5, activation="sigmoid"),
])

In [11]:
convolutional_model.compile(
optimizer='adam',
loss='categorical_crossentropy',
metrics=['accuracy'],
)
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)
convolutional_model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

# Evaluate the model on the test dataset
loss, accuracy = convolutional_model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

Epoch 1/50
65/65 [==============================] - 25s 357ms/step - loss: 3.7134 - accuracy: 0.3030 - val_loss: 1.4412 - val_accuracy: 0.4572
Epoch 2/50
65/65 [==============================] - 24s 371ms/step - loss: 1.1431 - accuracy: 0.5615 - val_loss: 1.3455 - val_accuracy: 0.4825
Epoch 3/50
65/65 [==============================] - 24s 370ms/step - loss: 1.0314 - accuracy: 0.5924 - val_loss: 1.2547 - val_accuracy: 0.5370
Epoch 4/50
65/65 [==============================] - 24s 370ms/step - loss: 0.8300 - accuracy: 0.6746 - val_loss: 1.2195 - val_accuracy: 0.5525
Epoch 5/50
65/65 [==============================] - 24s 368ms/step - loss: 0.8002 - accuracy: 0.6942 - val_loss: 1.2791 - val_accuracy: 0.5467
Epoch 6/50
65/65 [==============================] - 24s 370ms/step - loss: 0.6708 - accuracy: 0.7547 - val_loss: 1.2659 - val_accuracy: 0.5564
Epoch 7/50
65/65 [==============================] - 24s 371ms/step - loss: 0.5861 - accuracy: 0.7864 - val_loss: 1.2864 - val_accuracy: 0.5623

Como podemos ver o resultado da rede Inception3 também não é satisfatório e ainda pior que da rede VGG16. Isso ressalta a importância de termos outros mecanismos de aprendizado a serem usados ao longo do treinamento da rede.

### Testando o Mesmo modelo com regularização L1 e L2

Uma ótima ferramenta para melhorar o trainamento de um modelo é a regularização L1 e L2. Estes se tratam de duas funções de regularização que adicionam um termo para a função de perda, o que facilita a encontrar um melhor modelo.

In [12]:
from tensorflow.keras.regularizers import l1, l2, l1_l2

convolutional_model = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=1e-4, l2=1e-3)),  # L1/L2 regularization
    layers.Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=1e-4, l2=1e-3)),
    layers.Dense(5, activation='softmax')  # Output layer
])

In [13]:
convolutional_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)
convolutional_model.fit(train_ds, train_labels, epochs=30, validation_split=0.2, batch_size=32)

# Evaluate the model on the test dataset
loss, accuracy = convolutional_model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

Epoch 1/30
65/65 [==============================] - 28s 402ms/step - loss: 4.1665 - accuracy: 0.3297 - val_loss: 2.4443 - val_accuracy: 0.5428
Epoch 2/30
65/65 [==============================] - 25s 383ms/step - loss: 2.1579 - accuracy: 0.5795 - val_loss: 2.1549 - val_accuracy: 0.4786
Epoch 3/30
65/65 [==============================] - 25s 381ms/step - loss: 1.8310 - accuracy: 0.5792 - val_loss: 1.9116 - val_accuracy: 0.5233
Epoch 4/30
65/65 [==============================] - 25s 380ms/step - loss: 1.5279 - accuracy: 0.6429 - val_loss: 1.9445 - val_accuracy: 0.5175
Epoch 5/30
65/65 [==============================] - 25s 382ms/step - loss: 1.3743 - accuracy: 0.6775 - val_loss: 1.6578 - val_accuracy: 0.5603
Epoch 6/30
65/65 [==============================] - 25s 389ms/step - loss: 1.2151 - accuracy: 0.7304 - val_loss: 2.1159 - val_accuracy: 0.4728
Epoch 7/30
65/65 [==============================] - 25s 382ms/step - loss: 1.1839 - accuracy: 0.7437 - val_loss: 1.8050 - val_accuracy: 0.5311

Como podemos ver, mesmo com as regularizações, não conseguimos melhorar o resultado. Isso aponta para que o modelo não está realmente ajustando bem o problema, e que para melhorar o resultado, é preciso melhorar os dados de treinamento.

In [14]:
from tensorflow.keras.regularizers import l1, l2, l1_l2

convolutional_model = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=1e-4, l2=1e-3)),  # L1/L2 regularization
    layers.Dropout(0.1),
    layers.Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=1e-4, l2=1e-3)),
    layers.Dropout(0.05),
    layers.Dense(5, activation='softmax')  # Output layer
])

Ao Adicionar camadas de Dropout no modelo permitimos a retirada de alguns pesos que estão em ótimos locais, andarmos em direção a um ótimo global, o que deve acarretar em uma melhor performance no final do treinamento.

In [15]:
convolutional_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)
convolutional_model.fit(train_ds, train_labels, epochs=30, validation_split=0.2, batch_size=32)

# Evaluate the model on the test dataset
loss, accuracy = convolutional_model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

Epoch 1/30
65/65 [==============================] - 27s 383ms/step - loss: 5.2109 - accuracy: 0.2545 - val_loss: 2.7968 - val_accuracy: 0.4261
Epoch 2/30
65/65 [==============================] - 25s 379ms/step - loss: 2.6148 - accuracy: 0.4404 - val_loss: 2.2362 - val_accuracy: 0.4805
Epoch 3/30
65/65 [==============================] - 24s 378ms/step - loss: 2.1034 - accuracy: 0.5043 - val_loss: 1.9076 - val_accuracy: 0.5428
Epoch 4/30
65/65 [==============================] - 24s 376ms/step - loss: 1.8149 - accuracy: 0.5325 - val_loss: 1.8841 - val_accuracy: 0.4728
Epoch 5/30
65/65 [==============================] - 24s 375ms/step - loss: 1.7388 - accuracy: 0.5287 - val_loss: 1.7297 - val_accuracy: 0.5331
Epoch 6/30
65/65 [==============================] - 24s 374ms/step - loss: 1.5564 - accuracy: 0.5855 - val_loss: 1.6656 - val_accuracy: 0.5409
Epoch 7/30
65/65 [==============================] - 24s 374ms/step - loss: 1.4625 - accuracy: 0.6076 - val_loss: 1.6630 - val_accuracy: 0.5525